In [86]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
    raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


In [2]:
%matplotlib inline

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

/Users/twist/Downloads/take2/utils/visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/Users/twist/.pyenv/versions/3.6.5/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/twist/.pyenv/versions/3.6.5/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/twist/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/twist/.pyenv/versions/3.6.5/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/twist/.pyenv/versions/3.6.5/lib/python3.6/site-packages

In [4]:
PATH_TO_CKPT = 'output/frozen_inference_graph.pb'

In [5]:
PATH_TO_LABELS = 'data/object-detection.pbtxt'

In [6]:
NUM_CLASSES = 1

In [44]:
# load graph
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [45]:
# load labels
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [46]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

['test_images/00_032_008.jpg', 'test_images/00_032_020.jpg', 'test_images/00_032_021.jpg', 'test_images/00_032_009.jpg', 'test_images/00_032_023.jpg', 'test_images/00_032_022.jpg', 'test_images/00_032_026.jpg', 'test_images/00_032_027.jpg', 'test_images/00_032_025.jpg', 'test_images/00_032_019.jpg', 'test_images/00_032_018.jpg', 'test_images/00_032_024.jpg', 'test_images/00_032_001.jpg', 'test_images/00_032_015.jpg', 'test_images/00_032_014.jpg', 'test_images/00_032_000.jpg', 'test_images/00_032_016.jpg', 'test_images/00_032_002.jpg', 'test_images/00_032_003.jpg', 'test_images/00_032_017.jpg', 'test_images/00_032_013.jpg', 'test_images/00_032_007.jpg', 'test_images/00_032_006.jpg', 'test_images/00_032_012.jpg', 'test_images/00_032_004.jpg', 'test_images/00_032_010.jpg', 'test_images/00_032_011.jpg', 'test_images/00_032_005.jpg']


In [48]:
def run_inference_for_single_image(image, sess):
    # Get handles to input and output tensors
    ops = tf.get_default_graph().get_operations()
    all_tensor_names = {output.name for op in ops for output in op.outputs}
    tensor_dict = {}
    for key in [
        'num_detections', 'detection_boxes', 'detection_scores',
        'detection_classes', 'detection_masks'
    ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
            tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
    if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
    image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

    # Run inference
    output_dict = sess.run(tensor_dict,
                           feed_dict={image_tensor: np.expand_dims(image, 0)})

    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
        'detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [59]:
#returns tuple (n_arr, score)= numpy array cropped head and classification score
def cut_head(image_path, session):
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, session)
    im_width, im_height = image.size
    detection_boxes = output_dict['detection_boxes']
    detection_classes = output_dict['detection_classes']
    detection_scores = output_dict['detection_scores']
    score = detection_scores[0]
    print("highest score = {}".format(detection_scores[0]))
    box = detection_boxes[0]
    ymin, xmin, ymax, xmax = box
    (left, right, top, bottom) = (int(xmin * im_width), int(xmax * im_width),
                                  int(ymin * im_height), int(ymax * im_height))
    
    area = (left, top, right, bottom)
    print("area = {}".format(area))
    # Visualization of the results of a detection.
#     print(output_dict['detection_classes'])
#     print(output_dict['detection_boxes'])
#     print('s============s')
#     print('s============s')
#     print('s============s')
#     print('s============s')
#     print(output_dict['detection_scores'])
#     draw = ImageDraw.Draw(image)
#     draw.line([(left, top), (left, bottom), (right, bottom),
#              (right, top), (left, top)], width=4, fill='red')
#     print(type(image))
    cropped = image.crop(area)
    n_arr = np.array(cropped)
#     print(n)
#     w1, h1 = image.size
#     w2, h2 = cropped.size
#     print(type(cropped))
#     print(image.size)
#     print(cropped.size)
#     n = np.asarray(cropped)
#     cropped.show()
#     print(n)
#     print(cropped)
#     cropped.save('test1.jpg')
#     from IPython.display import display
    
#     plt.figure(figsize=IMAGE_SIZE)
#     plt.imshow(cropped)
#     vis_util.visualize_boxes_and_labels_on_image_array(
#         image_np,
#         output_dict['detection_boxes'],
#         output_dict['detection_classes'],
#         output_dict['detection_scores'],
#         category_index,
#         instance_masks=output_dict.get('detection_masks'),
#         use_normalized_coordinates=True,
#         line_thickness=8)
#     plt.imshow(image)
    
    return (n_arr, score, area)

In [94]:
def getAllImagePath(path, data):
    if(os.path.isfile(path)):
        if(not path.split("/")[-1].startswith(".")):
            data.append(path)
    else:
        list = os.listdir(path)
        for x in list:
            getAllImagePath(path+ "/"+ x, data)

In [102]:
PATH_TO_TEST_IMAGES_DIR = 'test_rgb'
TEST_IMAGE_PATHS = []
paths = getAllImagePath(PATH_TO_TEST_IMAGES_DIR, TEST_IMAGE_PATHS)

# for f in os.listdir(PATH_TO_TEST_IMAGES_DIR):
#     TEST_IMAGE_PATHS.append(os.path.join(PATH_TO_TEST_IMAGES_DIR, f))
#TEST_IMAGE_PATHS = ["test_images/frame779.jpg"]
print(TEST_IMAGE_PATHS)

# Size, in inches, of the output images.
# IMAGE_SIZE = (12, 8)


['test_rgb/03/023/03_023_009.jpg', 'test_rgb/03/023/03_023_008.jpg', 'test_rgb/03/023/03_023_020.jpg', 'test_rgb/03/023/03_023_018.jpg', 'test_rgb/03/023/03_023_019.jpg', 'test_rgb/03/023/03_023_014.jpg', 'test_rgb/03/023/03_023_000.jpg', 'test_rgb/03/023/03_023_001.jpg', 'test_rgb/03/023/03_023_015.jpg', 'test_rgb/03/023/03_023_003.jpg', 'test_rgb/03/023/03_023_017.jpg', 'test_rgb/03/023/03_023_016.jpg', 'test_rgb/03/023/03_023_002.jpg', 'test_rgb/03/023/03_023_006.jpg', 'test_rgb/03/023/03_023_012.jpg', 'test_rgb/03/023/03_023_013.jpg', 'test_rgb/03/023/03_023_007.jpg', 'test_rgb/03/023/03_023_011.jpg', 'test_rgb/03/023/03_023_005.jpg', 'test_rgb/03/023/03_023_004.jpg', 'test_rgb/03/023/03_023_010.jpg', 'test_rgb/03/015/03_015_002.jpg', 'test_rgb/03/015/03_015_003.jpg', 'test_rgb/03/015/03_015_001.jpg', 'test_rgb/03/015/03_015_000.jpg', 'test_rgb/03/015/03_015_004.jpg', 'test_rgb/03/015/03_015_005.jpg', 'test_rgb/03/015/03_015_007.jpg', 'test_rgb/03/015/03_015_006.jpg', 'test_rgb/03/

In [103]:

for image_path in TEST_IMAGE_PATHS:
    img_n_arr, score = cut_head(image_path)
#     plt.imshow(img_n_arr)
    fname = image_path.split("/")[1]
    fout = os.path.join("test_images_heads", fname)
    Image.fromarray(img_n_arr).save(fout)

TypeError: cut_head() missing 1 required positional argument: 'session'

In [104]:
from keras.models import load_model
from keras.applications.mobilenet import preprocess_input
import numpy as np


def wraper(img_array, model):
    
    img = np.expand_dims(img_array, axis=0).astype('float32')
    img = preprocess_input(img)
    
    x = model.predict(img)
    return x

In [105]:
import cv2

output_list = []

length = len(TEST_IMAGE_PATHS)
to_generate = 100 #3000
print("generating for {} out of {} photos".format(to_generate, length))
i = 0

print("loading model")
with detection_graph.as_default():
    with tf.Session() as sess:
        with CustomObjectScope({'relu6': keras.applications.mobilenet.relu6,'DepthwiseConv2D': keras.applications.mobilenet.DepthwiseConv2D}):   
            model = load_model("model-31-07")
            print("model has been loaded")
            for image_path in TEST_IMAGE_PATHS[1000:1600]:
                img_n_arr, score, (left, top, right, bottom) = cut_head(image_path, sess)
                img = cv2.resize(img_n_arr, (128, 128))
            #   plt.imshow(img_n_arr)
#                 fname = image_path.split("/")[-1]
            #   fout = os.path.join("test_images_heads", fname)
            #   Image.fromarray(img_n_arr).save(fout)
                sincos = wraper(img, model)
                print(sincos)
                i += 1
                print("{}/{}".format(i, to_generate))
                sinx = sincos[0][0]
                cosx = sincos[0][1]
                row = (image_path, score, left, top, right, bottom, sinx, cosx)
                output_list.append(row)
                

generating for 100 out of 4010 photos
loading model
model has been loaded
highest score = 0.7786694765090942
area = (224, 127, 320, 212)
[[0.6889556  0.18336311]]
1/100
highest score = 0.3103201687335968
area = (265, 187, 317, 240)
[[0.8783972 0.6121855]]
2/100
highest score = 0.6922531127929688
area = (198, 78, 280, 152)
[[0.558499   0.09386404]]
3/100
highest score = 0.8534424901008606
area = (161, 32, 220, 91)
[[ 0.6617627  -0.08676055]]
4/100
highest score = 0.1276523321866989
area = (0, 173, 107, 234)
[[ 0.8895807  -0.11487839]]
5/100
highest score = 0.15147486329078674
area = (247, 38, 318, 131)
[[ 0.46654305 -0.5845189 ]]
6/100
highest score = 0.27167564630508423
area = (124, 2, 230, 59)
[[ 0.21572551 -0.32780486]]
7/100
highest score = 0.3528845012187958
area = (80, 0, 134, 47)
[[ 0.12514454 -0.2921562 ]]
8/100
highest score = 0.1709609031677246
area = (0, 76, 110, 139)
[[ 0.33949888 -0.6049497 ]]
9/100
highest score = 0.15915270149707794
area = (7, 0, 81, 38)
[[ 0.4344027  -0.

highest score = 0.5364112257957458
area = (77, 44, 136, 109)
[[ 0.37231895 -0.4598456 ]]
86/100
highest score = 0.16557742655277252
area = (0, 42, 95, 206)
[[-0.30931556 -0.5896052 ]]
87/100
highest score = 0.2803080379962921
area = (0, 25, 34, 84)
[[-0.15640382 -0.76103663]]
88/100
highest score = 0.3649986684322357
area = (42, 54, 103, 111)
[[1.0199019  0.12035152]]
89/100
highest score = 0.8234566450119019
area = (114, 46, 177, 105)
[[ 0.69456375 -0.11991951]]
90/100
highest score = 0.13982340693473816
area = (0, 176, 112, 232)
[[ 0.8192659  -0.03268494]]
91/100
highest score = 0.8719859719276428
area = (186, 175, 315, 237)
[[ 0.21834354 -0.1105838 ]]
92/100
highest score = 0.24072164297103882
area = (123, 59, 231, 129)
[[ 0.06252922 -0.37248406]]
93/100
highest score = 0.4279721975326538
area = (182, 159, 299, 232)
[[0.23106922 0.05691537]]
94/100
highest score = 0.4964332580566406
area = (202, 114, 290, 191)
[[ 0.32488626 -0.37722784]]
95/100
highest score = 0.43366825580596924
ar

highest score = 0.4684717655181885
area = (33, 0, 91, 53)
[[ 0.10442793 -0.5022347 ]]
171/100
highest score = 0.7922101616859436
area = (95, 15, 152, 70)
[[ 0.10367004 -0.25707024]]
172/100
highest score = 0.7591615915298462
area = (229, 143, 320, 215)
[[ 0.6288381 -0.4371704]]
173/100
highest score = 0.7143722176551819
area = (217, 143, 320, 210)
[[ 0.3474595  -0.31315875]]
174/100
highest score = 0.4330643117427826
area = (184, 98, 296, 174)
[[ 0.23001224 -0.27745032]]
175/100
highest score = 0.6253502368927002
area = (224, 88, 286, 147)
[[0.9385759 0.2162848]]
176/100
highest score = 0.4860304892063141
area = (231, 138, 319, 208)
[[ 0.33458766 -0.32513663]]
177/100
highest score = 0.8046990036964417
area = (144, 50, 204, 113)
[[ 0.688058  -0.2442747]]
178/100
highest score = 0.41840964555740356
area = (203, 121, 314, 195)
[[ 0.38150895 -0.03056904]]
179/100
highest score = 0.2996666133403778
area = (271, 149, 320, 210)
[[ 0.70630205 -0.3526295 ]]
180/100
highest score = 0.6876906752

highest score = 0.8605355620384216
area = (226, 130, 320, 217)
[[ 0.28056398 -0.761504  ]]
256/100
highest score = 0.15118663012981415
area = (149, 0, 320, 165)
[[ 0.15803057 -0.5578252 ]]
257/100
highest score = 0.1652962565422058
area = (28, 24, 305, 240)
[[-0.24292724 -0.62008756]]
258/100
highest score = 0.749144434928894
area = (212, 119, 311, 221)
[[ 0.15276621 -0.8661982 ]]
259/100
highest score = 0.3913511037826538
area = (188, 160, 304, 236)
[[ 0.34787077 -0.0395395 ]]
260/100
highest score = 0.8222434520721436
area = (213, 98, 296, 210)
[[ 0.01386509 -0.8675967 ]]
261/100
highest score = 0.8813096284866333
area = (214, 139, 297, 235)
[[ 0.39287278 -0.6885094 ]]
262/100
highest score = 0.39886927604675293
area = (96, 115, 194, 192)
[[ 0.61626333 -0.35172424]]
263/100
highest score = 0.4131366014480591
area = (46, 130, 104, 187)
[[0.7493526 0.1921264]]
264/100
highest score = 0.8083311319351196
area = (219, 157, 308, 237)
[[ 0.32312974 -0.32119346]]
265/100
highest score = 0.65

highest score = 0.5789708495140076
area = (243, 109, 305, 164)
[[0.8875379  0.15460965]]
340/100
highest score = 0.18305809795856476
area = (44, 12, 161, 85)
[[ 0.06369698 -0.26436564]]
341/100
highest score = 0.3971177041530609
area = (0, 1, 37, 59)
[[-0.27769178 -0.54991746]]
342/100
highest score = 0.4868760108947754
area = (252, 126, 313, 187)
[[ 0.84628433 -0.02920337]]
343/100
highest score = 0.5000450015068054
area = (134, 19, 221, 101)
[[-0.04864487 -0.5521215 ]]
344/100
highest score = 0.35061609745025635
area = (269, 151, 320, 211)
[[ 0.70662606 -0.05894714]]
345/100
highest score = 0.6885444521903992
area = (262, 139, 320, 197)
[[0.8008008  0.11105846]]
346/100
highest score = 0.6397325396537781
area = (54, 6, 116, 59)
[[0.6883992  0.16380143]]
347/100
highest score = 0.14664970338344574
area = (0, 0, 70, 39)
[[ 0.42027232 -0.07605857]]
348/100
highest score = 0.6534148454666138
area = (199, 56, 292, 135)
[[ 0.11501252 -0.7467187 ]]
349/100
highest score = 0.5523011088371277

highest score = 0.27204835414886475
area = (205, 179, 316, 240)
[[0.7364328  0.10588452]]
425/100
highest score = 0.24633678793907166
area = (218, 198, 311, 240)
[[0.58141774 0.26573426]]
426/100
highest score = 0.6814213991165161
area = (5, 89, 69, 148)
[[ 0.69746697 -0.02079527]]
427/100
highest score = 0.1306840181350708
area = (0, 47, 84, 202)
[[-0.28901485 -0.6437809 ]]
428/100
highest score = 0.22362685203552246
area = (17, 92, 146, 187)
[[0.3376247  0.04821206]]
429/100
highest score = 0.458176851272583
area = (202, 105, 290, 187)
[[ 0.05604966 -0.49179944]]
430/100
highest score = 0.4283243715763092
area = (251, 156, 320, 238)
[[ 0.5141957  -0.25409558]]
431/100
highest score = 0.6278320550918579
area = (190, 178, 307, 240)
[[ 0.58001786 -0.19098152]]
432/100
highest score = 0.7259542346000671
area = (181, 171, 312, 232)
[[0.39339796 0.2666524 ]]
433/100
highest score = 0.4985387325286865
area = (87, 67, 180, 151)
[[-0.24918826 -0.46445256]]
434/100
highest score = 0.3829761743

highest score = 0.5522031784057617
area = (185, 134, 297, 240)
[[ 0.420872   -0.55669683]]
510/100
highest score = 0.13291212916374207
area = (0, 0, 77, 131)
[[-0.0012257 -0.6057846]]
511/100
highest score = 0.4172343313694
area = (73, 125, 139, 179)
[[0.8060753  0.37639478]]
512/100
highest score = 0.7619838118553162
area = (149, 61, 212, 117)
[[0.86615855 0.27580702]]
513/100
highest score = 0.6397812366485596
area = (182, 132, 302, 240)
[[ 0.24009375 -0.5806347 ]]
514/100
highest score = 0.6477018594741821
area = (184, 127, 289, 220)
[[ 0.6258668  -0.42062432]]
515/100
highest score = 0.3723633587360382
area = (196, 97, 257, 156)
[[0.48410115 0.18866122]]
516/100
highest score = 0.4578842520713806
area = (81, 142, 151, 203)
[[0.7540209  0.14318626]]
517/100
highest score = 0.6075753569602966
area = (189, 132, 298, 225)
[[ 0.57672924 -0.35610434]]
518/100
highest score = 0.7227922677993774
area = (165, 112, 248, 214)
[[ 0.49847034 -0.55459124]]
519/100
highest score = 0.4106653332710

highest score = 0.15216970443725586
area = (0, 0, 121, 86)
[[ 0.28530535 -0.52065337]]
595/100
highest score = 0.4906238317489624
area = (254, 159, 313, 226)
[[ 0.7594161  -0.00386709]]
596/100
highest score = 0.15268364548683167
area = (22, 151, 102, 192)
[[0.14646275 0.06371868]]
597/100
highest score = 0.8764158487319946
area = (188, 148, 278, 214)
[[ 0.12445943 -0.51272994]]
598/100
highest score = 0.5617436766624451
area = (177, 155, 264, 226)
[[ 0.44076362 -0.43155316]]
599/100
highest score = 0.7138293981552124
area = (175, 145, 261, 230)
[[-0.00312058 -0.5815697 ]]
600/100


In [83]:
output_list

[('test_rgb/03/023/03_023_009.jpg',
  0.16206455,
  20,
  0,
  99,
  39,
  0.23554617,
  -0.066188306),
 ('test_rgb/03/023/03_023_008.jpg',
  0.26809335,
  45,
  4,
  135,
  78,
  0.43281615,
  -0.3851219),
 ('test_rgb/03/023/03_023_020.jpg',
  0.15098295,
  237,
  99,
  320,
  139,
  0.94692427,
  0.110184334),
 ('test_rgb/03/023/03_023_018.jpg',
  0.2462485,
  263,
  14,
  320,
  78,
  0.42026705,
  -0.015588567),
 ('test_rgb/03/023/03_023_019.jpg',
  0.15032065,
  0,
  0,
  117,
  85,
  0.4611366,
  -0.22461993),
 ('test_rgb/03/023/03_023_014.jpg',
  0.18988882,
  113,
  0,
  182,
  96,
  -0.04553189,
  -0.17512044),
 ('test_rgb/03/023/03_023_000.jpg',
  0.14398366,
  281,
  38,
  320,
  97,
  -0.65011585,
  -0.66332495),
 ('test_rgb/03/023/03_023_001.jpg',
  0.15248625,
  0,
  0,
  121,
  86,
  0.55224043,
  -0.098058574),
 ('test_rgb/03/023/03_023_015.jpg',
  0.19974302,
  141,
  0,
  228,
  91,
  0.258079,
  -0.013609476),
 ('test_rgb/03/023/03_023_003.jpg',
  0.41060865,
  181,


In [79]:
!touch output.csv

In [100]:
import csv
with open('output.csv', 'w', newline='') as csvfile:
    fieldnames = ['image_path', 'score', 'left', 'top', 'right', 'bottom', 'sinx', 'cosx']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in output_list:
        writer.writerow({'image_path': row[0], 'score': row[1], 'left': row[2], 'top': row[3], 
                         'right': row[4], 'bottom': row[5], 'sinx': row[6], 'cosx': row[7]})

In [101]:
!head -100 output.csv

In [96]:
!rm output.csv